In [5]:
import subprocess
import time
import psutil
import os

# Levantar el contenedor en segundo plano
subprocess.run(["docker", "compose", "-f", "scripts/Docker/compose.yml", "up", "-d"])
print("Docker levantado")

# Esperar a que arranque el servicio
time.sleep(5)

# Obtener uso de CPU y RAM total del sistema
cpu_percent = psutil.cpu_percent(interval=1)
ram_info = psutil.virtual_memory()
ram_used = ram_info.used / (1024**3)  # GB

# Ver uso de disco de la imagen del emulador
image_name = "emuladorjs"
inspect = subprocess.run(
    ["docker", "image", "inspect", image_name, "--format={{.Size}}"],
    capture_output=True,
    text=True
)

if inspect.returncode == 0 and inspect.stdout.strip().isdigit():
    image_size_bytes = int(inspect.stdout.strip())
    image_size_gb = image_size_bytes / (1024**3)
    image_size_str = f"{image_size_gb:.2f} GB"
else:
    image_size_str = "No se pudo obtener"

# Parar el contenedor
subprocess.run(["docker", "compose", "-f", "scripts/Docker/compose.yml", "down"])

# Mostrar resultados
print(f"RAM usada (sistema): {ram_used:.2f} GB")
print(f"Uso de CPU (1s): {cpu_percent}%")
print(f"Tamaño imagen Docker '{image_name}': {image_size_str}")


Docker levantado
RAM usada (sistema): 13.80 GB
Uso de CPU (1s): 28.0%
Tamaño imagen Docker 'emuladorjs': No se pudo obtener


In [7]:
def medir_tiempo_arranque_docker(imagen="emulatorjs/emuladorjs", nombre_contenedor="emulador_benchmark"):
    # Limpiar contenedor previo si existe
    subprocess.run(f"docker rm -f {nombre_contenedor} || true", shell=True)
    
    inicio = time.time()
    # Arrancar contenedor
    subprocess.run(f"docker run --name {nombre_contenedor} -d -p 8080:80 {imagen}", shell=True)
    
    # Esperar unos segundos para que levante
    time.sleep(5)
    
    fin = time.time()
    
    # Parar y borrar contenedor para limpieza
    subprocess.run(f"docker rm -f {nombre_contenedor}", shell=True)
    
    return fin - inicio

# Prueba la función
tiempo_docker = medir_tiempo_arranque_docker()
print(f"Tiempo de arranque Docker: {tiempo_docker:.2f} segundos")


Tiempo de arranque Docker: 6.91 segundos


In [20]:
import time
import os
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def cpu_benchmark(iterations=10**7):
    print("🔧 Benchmark CPU...")
    start = time.time()
    x = 0
    for i in range(iterations):
        x += i*i
    end = time.time()
    print(f"Tiempo CPU: {end - start:.2f} segundos")

def disk_benchmark(file_path="testfile.tmp", size_mb=100):
    print("💾 Benchmark Disco...")

    data = b"0" * (1024 * 1024)  # 1 MB chunk
    chunks = size_mb

    # Escritura
    start = time.time()
    with open(file_path, "wb") as f:
        for _ in range(chunks):
            f.write(data)
    end = time.time()
    print(f"Escritura {size_mb}MB: {end - start:.2f} segundos")

    # Lectura
    start = time.time()
    with open(file_path, "rb") as f:
        while f.read(1024 * 1024):
            pass
    end = time.time()
    print(f"Lectura {size_mb}MB: {end - start:.2f} segundos")

    # Borrar archivo
    os.remove(file_path)

def network_benchmark_debug(url="http://ipv4.download.thinkbroadband.com/100MB.zip"):
    print("🌐 Benchmark Red...")
    start = time.time()
    try:
        r = requests.get(url, stream=True, verify=False)
        total_bytes = 0
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                total_bytes += len(chunk)
                #print(f"Chunk recibido: {len(chunk)} bytes, total acumulado: {total_bytes} bytes")
        r.close()
        elapsed = time.time() - start
        total_mb = total_bytes / (1024 * 1024)
        print(f"Descargados {total_mb:.2f} MB en {elapsed:.2f} segundos")
    except requests.exceptions.RequestException as e:
        print(f"Error en la descarga: {e}")

# Ejecutar benchmarks
cpu_benchmark()
disk_benchmark()
network_benchmark_debug()


🔧 Benchmark CPU...
Tiempo CPU: 0.90 segundos
💾 Benchmark Disco...
Escritura 100MB: 0.05 segundos
Lectura 100MB: 0.06 segundos
🌐 Benchmark Red...
Descargados 100.00 MB en 32.18 segundos
